# Baseline **LSTM** Frame Predictor

A companion to the Transformer notebook — shares the same dataset loading,
section layout, and training loop so you can compare apples‑to‑apples.

<small>*(Sections are numbered & boxed exactly like the Transformer notebook.)*</small>

In [4]:
import tensorflow as tf
import numpy as np

# ╔════════════════════╗
# ║ 1. Load  Dataset  ║
# ╚════════════════════╝
DATASET_PATH = "../processed_data/transformer_dataset"  # adjust if needed

ds = (tf.data.Dataset.load(DATASET_PATH)
      .shuffle(4096)
      .batch(64)
      .prefetch(tf.data.AUTOTUNE))

# Peek at one batch
for X, y in ds.take(1):
    print("X shape:", X.shape)
    print("y shape:", y.shape)
    break

X shape: (64, 100, 46)
y shape: (64, 46)


In [5]:
import matplotlib.pyplot as plt

# ╔═══════════════════╗
# ║ 2. Sanity checks ║
# ╚═══════════════════╝
NUM_FEATS = 46
MAX_LEN   = 100

def sanity_check(x, y):
    assert x.shape == (MAX_LEN, NUM_FEATS)
    assert y.shape == (NUM_FEATS,)
    nonzero_rows = np.any(x.numpy() != 0, axis=1)
    pad = MAX_LEN - np.count_nonzero(nonzero_rows)
    print(f"Padding rows: {pad}")
    if np.allclose(y.numpy(), 0):
        print("⚠️ y is all zeros!")

for X_batch, y_batch in ds.take(1):
    sanity_check(X_batch[0], y_batch[0])
    break

Padding rows: 34


In [6]:
from tensorflow import keras
from tensorflow.keras import layers

# ╔══════════════════════╗
# ║ 3. LSTM Model Build ║
# ╚══════════════════════╝
# Hyper‑parameters
LSTM_UNITS = 128
EMBED_DIM  = 4        # position embedding (unused here — coords‑only baseline)

def mse_sum_xy(y_true, y_pred):
    diff = tf.reshape(y_true - y_pred, (-1, 23, 2))
    return tf.reduce_sum(tf.square(diff), axis=[1,2])

def build_lstm(max_len=MAX_LEN, num_feats=NUM_FEATS, lstm_units=LSTM_UNITS):
    seq_in = layers.Input(shape=(max_len, num_feats), name="sequence")
    x = layers.LSTM(lstm_units, name="lstm_backbone")(seq_in)
    x = layers.Dense(lstm_units, activation="relu", name="dense_relu")(x)
    out = layers.Dense(num_feats, name="pred_xy")(x)
    model = keras.Model(seq_in, out, name="Baseline_LSTM")
    model.compile(optimizer=keras.optimizers.Adam(1e-4),
                  loss=mse_sum_xy,
                  metrics=[keras.metrics.MeanAbsoluteError()])
    return model

model = build_lstm()
model.summary()

Model: "Baseline_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequence (InputLayer)           │ (None, 100, 46)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_backbone (LSTM)            │ (None, 128)            │        89,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_relu (Dense)              │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pred_xy (Dense)                 │ (None, 46)             │         5,934 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 112,046 (437.68 KB)

 Trainable params: 112,046 (437.68 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# ╔══════════════════════╗
# ║ 5. Compile & Train  ║
# ╚══════════════════════╝
EPOCHS = 15
val_split = 0.05
val_ds = ds.take(int(len(ds)*val_split))
train_ds = ds.skip(int(len(ds)*val_split))

history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, verbose=2)

Epoch 1/15
38050/38050 - 1884s - 50ms/step - loss: 0.0307 - mean_absolute_error: 0.0093 - val_loss: 6.9056e-04 - val_mean_absolute_error: 0.0024
Epoch 2/15
38050/38050 - 1995s - 52ms/step - loss: 4.0324e-04 - mean_absolute_error: 0.0019 - val_loss: 3.0334e-04 - val_mean_absolute_error: 0.0017
Epoch 3/15
38050/38050 - 5595s - 147ms/step - loss: 1.9981e-04 - mean_absolute_error: 0.0014 - val_loss: 1.9871e-04 - val_mean_absolute_error: 0.0015
Epoch 4/15


In [ ]:
# ╔═══════════════╗
# ║ 6. Evaluation ║
# ╚═══════════════╝
for X_batch, y_batch in val_ds.take(1):
    y_pred = model(X_batch)
    mse = tf.reduce_mean(tf.square(y_pred - y_batch))
    print("Validation MSE (batch):", mse.numpy())

In [ ]:
# ╔════════════════╗
# ║ 7. Curves Plot ║
# ╚════════════════╝
plt.figure()
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.title('Loss curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()